# Chapter 15 - 콘텍스트 관리자와 else 블록

## 이것 다음에 저것: if문 이외에서의 else 블록

if/else에서의 else라는 단어는 '그렇지 않으면'이라는 뜻인데 반해 그 외의 경우에서의 else는 '그러고 나면'이라는 뜻이다.<br>

- for/else: for 루프가 완전히 실행된 후에(break 문으로 중간에 멈추지 않고) else 블록이 실행된다.
- while/else: 조건식이 거짓이 되어 while 루프를 빠져나온 후에(break 문으로 중간에 멈추지 않고) else 블록이 실행된다.
- try/else: try 블록에서 예외가 발생하지 않을 때만 else 블록이 실행된다.

## 콘텍스트 관리자와 with 블록

In [1]:
class LookingGlass:
    def __enter__(self):
        import sys
        self.original_write = sys.stdout.write
        sys.stdout.write = self.reverse_write
        return 'JABBERWOCKY'
    
    def reverse_write(self, text):
        self.original_write(text[::-1])
        
    def __exit__(self, exc_type, exc_value, traceback):
        import sys
        sys.stdout.write = self.original_write
        if exc_type is ZeroDivisionError:
            print('Please DO NOT divide by zero!')
            return True

with LookingGlass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)
    
what

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


'JABBERWOCKY'

In [2]:
print('Back to normal.')

Back to normal.


In [3]:
manager = LookingGlass()
manager

In [4]:
monster = manager.__enter__()
monster == 'JABBERWOCKY'  # why 'True', not 'eurT'?

True

In [5]:
monster

'JABBERWOCKY'

In [6]:
manager.__exit__(None, None, None)

In [7]:
monster

'JABBERWOCKY'

## @contextmanager 사용하기

@contextmanager 데커레이터는 콘텍스트 관리자를 생성할 때 작성하는 틀에 박힌 코드를 줄여준다.<br>
\_\_enter\_\_()와 \_\_exit\_\_() 메서드를 가진 클래스 전체를 작성하는 대신 \_\_enter\_\_() 메서드가 반환할 것을 생성하는 yield문 하나를 가진 제너레이터만 구현하면 된다.

In [2]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    yield 'JABBERWOCKY'
    sys.stdout.write = original_write

In [3]:
with looking_glass() as what:
    print('Alice, Kitty and Snowdrop')
    print(what)

pordwonS dna yttiK ,ecilA
YKCOWREBBAJ


In [4]:
what

'JABBERWOCKY'

위 예제는 심각한 문제가 있는데, with 블록 안에서 예외가 발생하면 파이썬 인터프리터가 이 예외를 잡고<br>
looking_glass() 안에 있는 yield 표현식에서 다시 예외를 발생시킨다.<br>
그러나 그곳에는 예외 처리 코드가 없어서 looking_glass() 함수는 원래의 sys.stdout.write() 메서드를 복원하지 않고 중단한다.<br>
다음은 ZeroDivisionError 예외를 특별히 처리하는 코드이다.

In [5]:
import contextlib

@contextlib.contextmanager
def looking_glass():
    import sys
    original_write = sys.stdout.write
    
    def reverse_write(text):
        original_write(text[::-1])
        
    sys.stdout.write = reverse_write
    msg = ''
    try:
        yield 'JABBERWOCKY'
    except ZeroDivisionError:
        msg = 'Please DO NOT divide by zero!'
    finally:
        sys.stdout.write = original_write
        if msg:
            print(msg)